In [3]:
import pandas as pd
import os

# Defina o caminho onde seus arquivos CSV estão localizados
csv_directory = '.\\csv'

# Inicialize uma lista para armazenar os dataframes
dataframes = []

# Percorra todos os arquivos no diretório
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(csv_directory, filename)
        # Leia o arquivo CSV e adicione o dataframe à lista
        df = pd.read_csv(filepath, sep=';')

        # Remova as colunas 'Class' e 'Role %'
        df = df.drop(columns=['Class', 'Role %'])

        dataframes.append(df)

# Combine todos os dataframes em um único dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

# Salve o dataframe combinado em um novo arquivo CSV
combined_df.to_csv('Season12LeagueOfLegends.csv', sep=';', index=False)

print('Arquivos CSV combinados com sucesso!')


Arquivos CSV combinados com sucesso!


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Carregar o arquivo CSV
df = pd.read_csv('Season12LeagueOfLegends.csv', delimiter=';')

# Exibir o DataFrame original
print("DataFrame Original:")
print(df)

# Remover o símbolo '%' e converter as colunas para float
percent_columns = ['Win %', 'Pick %', 'Ban %']
for col in percent_columns:
    df[col] = df[col].str.rstrip('%').astype(float)

# Transformar colunas categóricas em variáveis dummy, exceto a variável alvo
df_transformed = pd.get_dummies(df, columns=['Role', 'Name'], dtype=int)

# Separar features (X) e variável alvo (y)
X = df_transformed.drop(columns=['Tier'])
y = df_transformed['Tier']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar colunas numéricas
numerical_columns = ['Score', 'Trend', 'Win %', 'Pick %', 'Ban %', 'KDA']
scaler = MinMaxScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Exibir os DataFrames transformados
print("\nFeatures de Treinamento:")
print(X_train)
print("\nVariável Alvo de Treinamento:")
print(y_train)
print("\nFeatures de Teste:")
print(X_test)
print("\nVariável Alvo de Teste:")
print(y_test)

# Combinar os dados de treinamento com suas variáveis alvo
train_data = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
test_data = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

# Exportar para CSV
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)


DataFrame Original:
        Name     Role Tier  Score  Trend   Win % Pick %   Ban %   KDA
0     Aatrox      TOP    A  58.25   6.52  49.97%  4.43%   2.03%  1.97
1       Ahri      MID    A  53.21  -0.24  49.93%  4.62%   1.04%  2.56
2      Akali      MID    S  65.30   6.51  48.59%  8.16%  12.88%  2.34
3      Akali      TOP    A  57.87   3.34  48.57%  4.24%  12.88%  2.04
4     Akshan      MID    S  59.85   0.65  51.46%  4.83%  21.91%  2.23
...      ...      ...  ...    ...    ...     ...    ...     ...   ...
5539   Ziggs      ADC    C  34.08  -4.45  49.99%  1.66%   0.43%  2.33
5540  Zilean      MID    A  53.09   4.02  54.04%  0.61%   1.59%  3.29
5541  Zilean  SUPPORT    A  49.49   3.80  50.96%  2.69%   1.81%  3.11
5542     Zoe      MID    A  55.13   5.52  48.44%  5.81%   2.01%  2.34
5543    Zyra  SUPPORT    B  46.65  -2.51  50.83%  3.20%   1.83%  2.13

[5544 rows x 9 columns]

Features de Treinamento:
         Score     Trend     Win %    Pick %     Ban %       KDA  Role_ADC  \
5145  0.440

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Carregar o arquivo CSV
df = pd.read_csv('Season12LeagueOfLegends.csv', delimiter=';')

# Remover o símbolo '%' e converter as colunas para float
percent_columns = ['Win %', 'Pick %', 'Ban %']
for col in percent_columns:
    df[col] = df[col].str.rstrip('%').astype(float)

# Transformar colunas categóricas em variáveis dummy, exceto a variável alvo
df_transformed = pd.get_dummies(df, columns=['Role', 'Name'], dtype=int)

# Separar features (X) e variável alvo (y)
X = df_transformed.drop(columns=['Tier'])
y = df_transformed['Tier']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalizar colunas numéricas
numerical_columns = ['Score', 'Trend', 'Win %', 'Pick %', 'Ban %', 'KDA']
scaler = MinMaxScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Treinar o modelo de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o modelo
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Exportar o modelo treinado
joblib.dump(clf, 'decision_tree_model.pkl')

# Combinar os dados de treinamento com suas variáveis alvo
train_data = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
test_data = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

# Exportar para CSV
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)



Classification Report:
              precision    recall  f1-score   support

           A       0.86      0.91      0.88       417
           B       0.84      0.79      0.81       423
           C       0.64      0.67      0.65       239
           D       0.79      0.79      0.79       175
         God       0.91      0.81      0.86       179
           S       0.79      0.81      0.80       231

    accuracy                           0.81      1664
   macro avg       0.80      0.80      0.80      1664
weighted avg       0.81      0.81      0.81      1664


Confusion Matrix:
[[379  22   0   0   0  16]
 [ 31 334  58   0   0   0]
 [  0  40 161  38   0   0]
 [  0   2  34 139   0   0]
 [  0   0   0   0 145  34]
 [ 30   0   0   0  15 186]]


In [6]:
from sklearn import tree
import graphviz
import joblib


# Carregar o modelo
model = joblib.load('decision_tree_model.pkl')

# Exportar a árvore de decisão para o formato DOT
dot_data = tree.export_graphviz(model, out_file=None, 
                                feature_names=X.columns,  
                                class_names=model.classes_,  
                                filled=True, rounded=True,  
                                special_characters=True)  

# Criar o gráfico da árvore
graph = graphviz.Source(dot_data)  

# Salvar o gráfico em um arquivo
graph.render("decision_tree")

# Ou exibir diretamente no notebook ou terminal
graph.view()


'decision_tree.pdf'